# 🌍 Open-Meteo - Data Engineering Project

The **Silver Layer** is the second stage in our Data Lake architecture following the Bronze Layer. Its primary goal is to clean, enrich, and standardize the raw data ingested and stored previously.

While the **Bronze Layer** stores raw, unvalidated data exactly as retrieved from external sources (such as the Open-Meteo API), the **Silver Layer** applies transformation logic to make this data:

- More structured and consistent,
- Ready for business-level processes,
- Easier to join and analyze across different domains.

### **🪙 Storage : Silver Layer**

> ⚙️ In this notebook, we will read the Bronze Delta tables related to weather (current, forecast, historical, geolocation, etc.), clean and enrich them, and store them back as **Delta Tables** in the **Silver layer** following best practices.

---

**Main Transformations Applied:**
1. Normalize naming conventions (column names, types, formatting).
2. Elimination or replacement of nulls.
3. Join datasets to enrich context (e.g., weather codes or city metadata).
4. Add new calculated columns.
5. Elimination of duplicates.


The resulting Silver datasets will be well-structured and reliable for analytics, machine learning, or to be passed to the Gold Layer for specific use cases.


In [1]:
#Import libraries
import sys
import os
#to get summarized informations about df, just used for investigation
from ydata_profiling import ProfileReport
# Add the path to the modules directory
my_current_loc = os.getcwd()
my_modules_dir = "/Users/focus_profond/UTN/Data_engineering/proyecto/UTN_data_engineering_project/Entrega_Final/Modules"
os.chdir(my_modules_dir)

#Importing personal modules
from DF_functions import *
from openmeteo_API import *

#Returning to the main directory
#my_main_dir = "/Users/focus_profond/UTN/Data_engineering/proyecto/UTN_data_engineering_project/Entrega"
os.chdir(my_current_loc)
os.chdir('../')
#print(os.getcwd())



Module exécuté dans : /Users/focus_profond/UTN/Data_engineering/proyecto/UTN_data_engineering_project/utn_env/bin/python


#### 🌍 **2.1 GEOCODING**
**🧹 Cleaning & Transformation Logic:**

We will apply the following transformations:

- **Drop irrelevant or unstable columns** (e.g., `postcodes`, etc.)
- **Rename columns** for clarity and consistency.
- **Convert data types** to ensure schema consistency (e.g., float for coordinates, string for identifiers).
- **Handle missing values** by replacing `nulls` with default values when appropriate (e.g., empty strings or `-1`).
- **Order the table**

>We will keep the same logic of the bronze layer, and merge the data. 

In [2]:
#PROFILING OF THE GEOLOCATION TABLE
#name_folder = 'Data/Bronze/OpenMeteo/Others/Geolocation'
#my_dt = DeltaTable(name_folder).to_pandas()
#profile = ProfileReport(my_dt)
#profile

In [3]:
# Load DeltaTable from the bronze layer.
name_folder = 'Data/Bronze/OpenMeteo/Others/Geolocation'
my_dt = DeltaTable(name_folder).to_pandas()

#drop  columns
my_dt = my_dt.drop(columns=['admin4', 'admin4_id','__index_level_0__'], axis=1)

#rename columns
my_dt = my_dt.rename(columns={"elevation":"elevation_m"})

#replace missing values
imputation_mapping = {
	      'population' : -1
          ,'admin1_id':-1
		  ,'admin2_id' : -1
          ,'admin3_id' : -1
          ,'admin1':'no_value'
          ,'admin2' : 'no_value'
          ,'admin3' : 'no_value'
	}
my_dt = my_dt.fillna(imputation_mapping)

#change types
conversion_mapping = {
    "admin1_id": "int64",
    "admin2_id": "int64",
    "admin3_id": "int64",
    "admin1_id": "int64",
    "population": "int64",
    "name": "string",
    "feature_code": "string",
    "country_code": "string",
    "timezone": "string",
    "country": "string",
    "admin1": "string",
    "admin2": "string",
    "admin3": "string",
    "probability": "int8",
    }
my_dt = my_dt.astype(conversion_mapping)

#order table
my_dt = order_table(my_dt, ['country', 'name'])



In [4]:
#STORING THE DATA
name_folder = 'Data/Silver/OpenMeteo/Others/Geolocation'
partition_cols = None
predicate = "target.id = source.id"
save_new_data_as_delta(my_dt,name_folder,predicate= predicate, partition_cols=partition_cols, layer = 'Silver', source= 'open-meteo-geoloc', author ='Augustin')

#Verifying the data of the silver layer
my_dt = DeltaTable(name_folder).to_pandas()
my_dt.head(6)

,id,name,latitude,longitude,elevation_m,feature_code,country_code,admin1_id,timezone,population,country_id,country,admin1,probability,admin2_id,admin2,admin3_id,admin3
0,2346995,Biu,10.61285,12.19458,762.0,PPLA2,NG,2346794,Africa/Lagos,95005,2328926,Nigeria,Borno State,1,8659843,Biu,9412374,Sulumthla
1,3418226,Bildudalur,65.68530,-23.59992,9999.0,PPL,IS,3426185,Atlantic/Reykjavik,170,2629691,Iceland,Westfjords,2,3426215,Vesturbyggð,-1,no_value
2,3435910,Buenos Aires,-34.61315,-58.37723,31.0,PPLC,AR,3433955,America/Argentina/Buenos_Aires,13076300,3865483,Argentina,Buenos Aires F.D.,1,-1,no_value,-1,no_value
3,2158177,Melbourne,-37.81400,144.96332,25.0,PPLA,AU,2145234,Australia/Melbourne,4246375,2077456,Australia,Victoria,1,7839805,Melbourne,-1,no_value
4,2800866,Brussels,50.85045,4.34878,28.0,PPLC,BE,2800867,Europe/Brussels,1019022,2802361,Belgium,Brussels Capital,1,6693370,Bruxelles-Capitale,2800865,Arrondissement Brussel-Hoofdstad
5,2790471,Namur,50.46690,4.86746,88.0,PPLA,BE,3337387,Europe/Brussels,106284,2802361,Belgium,Wallonia,1,-1,no_value,2790470,Arrondissement de Namur


#### 🌍 **2.2 WEATHER CODE**
**🧹 Cleaning & Transformation Logic:**

We will apply the following transformations:

- **Rename columns**
- **Split columns**
- **Explode rows**
- **Capitalize and strip column**
- **Convert data types** 
- **Order the table**


>We will keep the same logic of the bronze layer and overwrite the data. 

In [5]:
name_folder = 'Data/Bronze/OpenMeteo/Others/WeatherCode'
my_dt = DeltaTable(name_folder).to_pandas()

#rename columns
my_dt = my_dt.rename(columns={"Code": "weather_code"})
my_dt = my_dt.rename(columns={"Description": "description"})

# Split comma-separated values into lists and explode into rows
my_dt['weather_code'] = my_dt['weather_code'].astype(str).str.split(",")
my_dt['description'] = my_dt['description'].astype(str).str.split(",")
my_dt = my_dt.explode(['weather_code', 'description'], ignore_index=True)

# Clean and format Description column
my_dt['description'] = my_dt['description'].str.strip().str.capitalize()
my_dt['weather_code'] = my_dt['weather_code'].str.strip()

# Order table 
my_dt = order_table(my_dt, ['weather_code'])
#my_dt.head(8)


In [6]:
#STORING THE DATA
name_folder = 'Data/Silver/OpenMeteo/Others/WeatherCode'
mode = 'overwrite'
partition_cols = None
save_data_as_delta(my_dt,name_folder,mode=mode, partition_cols=partition_cols, layer = 'Silver', source = 'open-meteo-weathercode',author='Augustin')

#Verifying the data of the silver layer
my_dt = DeltaTable(name_folder).to_pandas()
#my_dt.info()
my_dt.head(4)

,weather_code,description
0,0,Clear sky
1,1,Mainly clear
2,2,Partly cloudy
3,3,Overcast


#### 🌍 **2.3 CURRENT WEATHER**
**🧹 Cleaning & Transformation Logic:**

We will apply the following transformations:

- **Delete columns**
- **Rename columns**
- **Convert data types** 
- **Order the table**


>We will keep the same logic of the bronze layer and overwrite the data. 

In [7]:
# Load DeltaTable 
name_folder = 'Data/Bronze/OpenMeteo/Current'
my_dt = DeltaTable(name_folder).to_pandas()

# Drop unnecessary column
my_dt.drop(columns=['__index_level_0__'], inplace=True)

# Rename columns to include units of measure
remapping = {
    "cloud_cover": "cloud_cover_inPercent", 
    "relative_humidity_2m": "relative_humidity_2m_inPercent", 
    "apparent_temperature": "apparent_temperature_C", 
    "precipitation_sum": "precipitation_sum_mm",
    "windspeed_10m": "windspeed_10m_kmh",
    "windgusts_10m": "windgusts_10m_kmh", 
    "temperature_2m": "temperature_2m_C",
    "snowfall": "snowfall_cm",
    "surface_pressure": "surface_pressure_hPa", 
    "shortwave_radiation_sum": "shortwave_radiation_sum_Wm2", 
    "precipitation": "precipitation_mm",
    "pressure_msl": "pressure_msl_hPa", 
    "showers": "shower_sum_mm",
    "rain": "rain_mm",
    "wind_speed_10m": "wind_speed_10m_kmh",
    "Date": "date",
    "Time": "time",
    "City": "city"
}
my_dt = my_dt.rename(columns=remapping)

# Change column types
my_dt["is_day"] = my_dt["is_day"].astype(bool)
my_dt["weather_code"] = my_dt["weather_code"].astype(int)
my_dt["weather_code"] = my_dt["weather_code"].astype(str)
my_dt['weather_code'] = my_dt['weather_code'].str.strip()
my_dt['longitude'] = my_dt['longitude'].astype(float)
my_dt['latitude'] = my_dt['latitude'].astype(float)
my_dt['date'] = pd.to_datetime(my_dt['date'], format='%Y-%m-%d')

#Ordering the table
my_dt = order_table(my_dt, ['city','date','time'])

# Display DataFrame info and preview
#my_dt.info()
#my_dt.head(6)

In [8]:
#IN ORDER TO UPDATE THE TYPE OF THE COLUMNS
#name_folder = 'Data/Silver/OpenMeteo/Current'
#write_deltalake(name_folder,my_dt,mode='overwrite',schema_mode='overwrite')

In [ ]:
#STORING THE DATA
name_folder = 'Data/Silver/OpenMeteo/Current'
predicate = "target.Date = source.Date AND target.Time = source.Time AND target.City = source.City"
partition_cols = "Date"
save_new_data_as_delta(my_dt,name_folder,predicate = predicate, partition_cols=partition_cols, layer = 'Silver', source = 'open-meteo-current',author='Augustin')

# Verifying the data of the silver layer
my_dt = DeltaTable(name_folder).to_pandas()
#my_dt.info()
my_dt.head(6)


,date,time,city,longitude,latitude,surface_pressure_hPa,snowfall_cm,is_day,wind_gusts_10m,pressure_msl_hPa,shower_sum_mm,apparent_temperature_C,wind_direction_10m,cloud_cover_inPercent,rain_mm,relative_humidity_2m_inPercent,wind_speed_10m_kmh,weather_code,precipitation_mm,temperature_2m_C
0,2025-04-12,23:39,Barcelona,2.15899,41.38879,1015.924927,0.0,True,31.680000,1021.200012,0.0,16.251104,128.659836,52.0,0.0,73.0,9.220499,2,0.0,16.950001
1,2025-04-12,23:39,Biu,12.19458,10.61285,926.992493,0.0,False,18.359999,1010.099976,0.0,22.967268,41.185837,0.0,0.0,24.0,7.653705,0,0.0,25.600000
2,2025-04-12,23:39,Boston,-71.05977,42.35843,1013.312744,0.0,False,47.519997,1015.700012,0.0,-1.700215,13.736293,100.0,0.0,92.0,16.676977,3,0.0,2.622000
3,2025-04-12,23:39,Brussels,4.34878,50.85045,1024.516846,0.0,True,28.080000,1027.800049,0.0,11.576593,342.000000,98.0,0.0,44.0,12.599999,3,0.0,15.050000
4,2025-04-12,23:39,Buenos Aires,-58.37723,-34.61315,1016.168091,0.0,True,37.439999,1018.400024,0.0,22.793194,69.717354,83.0,0.0,73.0,17.654688,3,0.0,22.650000
5,2025-04-12,23:39,Chicago,-87.65005,41.85003,996.988098,0.0,True,14.759999,1019.000000,0.0,1.370000,47.385983,100.0,0.0,90.0,12.229406,3,0.0,4.726000


#### 🌍 **2.4 FORECAST DAILY**
**🧹 Cleaning & Transformation Logic:**

We will apply the following transformations:

- **Delete columns**
- **Rename columns**
- **Compute new columns** (duration in hours and in minutes)
- **Convert data types** 
- **Order the table**


>We will keep the same logic of the bronze layer and merge the data. 

In [10]:
#UV INDEX : create an index to save this dataframe.

#   0 to 2	You can safely enjoy being outside!
#   3 to 7	Seek shade during midday hours! Slip on a shirt, slop on sunscreen and slap on hat!
#   8 and above	Avoid being outside during midday hours! Make sure you seek shade! Shirt, sunscreen and hat are a must!

# https://www.who.int/news-room/questions-and-answers/item/radiation-the-ultraviolet-(uv)-index

In [11]:
# Load DeltaTable 
name_folder = 'Data/Bronze/OpenMeteo/Forecast/Daily'
my_dt = DeltaTable(name_folder).to_pandas()


# Convert date columns to datetime format
my_dt['Requested_Date'] = pd.to_datetime(my_dt['Requested_Date'], format='%Y-%m-%d')
my_dt['forecast_day'] = pd.to_datetime(my_dt['forecast_day'], format='%Y-%m-%d')
# Change column types
my_dt['longitude'] = my_dt['longitude'].astype(float)
my_dt['latitude'] = my_dt['latitude'].astype(float)
my_dt["weather_code"] = my_dt["weather_code"].astype(int)
my_dt["weather_code"] = my_dt["weather_code"].astype(str)
my_dt['weather_code'] = my_dt['weather_code'].str.strip()

# Rename columns to include units of measure
remapping = {
    "temperature_2m_max": "temperature_2m_max_C",
    "temperature_2m_min": "temperature_2m_min_C",
    "apparent_temperature_max": "apparent_temperature_max_C",
    "apparent_temperature_min": "apparent_temperature_min_C",
    "precipitation_sum_mm": "precipitation_sum_mm",
    "rain_sum": "rain_sum_mm",
    "showers_sum": "showers_sum_mm",
    "snowfall_sum": "snowfall_sum_cm",
    "precipitation_hours": "precipitation_hours_h",
    "precipitation_probability_max": "precipitation_probability_max_inPercent",
    "sunshine_duration": "sunshine_duration_seconds",
    "daylight_duration": "daylight_duration_seconds",
    "wind_speed_10m_max": "wind_speed_10m_max_kmh",
    "wind_gusts_10m_max": "wind_gusts_10m_max_kmh",
    "wind_direction_10m_dominant": "wind_direction_10m_dominant_deg",
    "shortwave_radiation_sum": "shortwave_radiation_sum_MJm2",
    "et0_fao_evapotranspiration": "et0_fao_evapotranspiration_mm",
    "uv_index_max": "uv_index_max_index",
    "uv_index_clear_sky_max": "uv_index_clear_sky_max_index",
    "Requested_Date": 'requested_date',
    'City': 'city'
}
my_dt = my_dt.rename(columns=remapping)

# Drop irrelevant columns
my_dt.drop(columns=['sunrise', 'sunset','__index_level_0__'], inplace=True)

# Convert duration columns to minutes and hours
my_dt['sunshine_duration_minutes'] = my_dt['sunshine_duration_seconds'] / 60
my_dt['sunshine_duration_hours'] = my_dt['sunshine_duration_seconds'] / 3600
my_dt['daylight_duration_minutes'] = my_dt['daylight_duration_seconds'] / 60
my_dt['daylight_duration_hours'] = my_dt['daylight_duration_seconds'] / 3600


#ordering the table
my_dt = order_table(my_dt, ['requested_date','city','forecast_day'])

# Display DataFrame info and preview
#my_dt.info()
#my_dt.head(6)

In [12]:
#IN ORDER TO UPDATE THE TYPE OF THE COLUMNS
#name_folder = 'Data/Silver/OpenMeteo/Forecast/Daily'
#write_deltalake(name_folder,my_dt,mode='overwrite',schema_mode='overwrite')

In [13]:
#STORING THE DATA
name_folder = 'Data/Silver/OpenMeteo/Forecast/Daily'
predicate = """target.requested_date = source.requested_date AND target.city = source.city and target.forecast_day = source.forecast_day """
partition_cols = ["requested_date"]

save_new_data_as_delta(my_dt,name_folder,predicate= predicate, partition_cols=partition_cols,layer = 'Silver', source= 'open-meteo-forecast-daily', author ='Augustin')


# Verifying the data of the silver layer
my_dt = DeltaTable(name_folder).to_pandas()
#my_dt.info()
my_dt.head(6)

,requested_date,city,forecast_day,weather_code,apparent_temperature_min_C,sunshine_duration_seconds,rain_sum_mm,precipitation_probability_max_inPercent,shortwave_radiation_sum_MJm2,temperature_2m_max_C,...,apparent_temperature_max_C,precipitation_sum,precipitation_hours_h,wind_direction_10m_dominant_deg,latitude,longitude,sunshine_duration_minutes,sunshine_duration_hours,daylight_duration_minutes,daylight_duration_hours
0,2025-04-12,Biu,2025-04-15 00:00:00+00:00,3,22.393799,39693.480469,0.0,3.0,27.260000,38.573498,...,36.625225,0.0,0.0,66.236183,10.61285,12.19458,661.557983,11.025967,742.186218,12.369770
1,2025-04-12,Biu,2025-04-16 00:00:00+00:00,2,23.345686,42075.976562,0.0,10.0,28.160000,38.823498,...,36.508705,0.0,0.0,72.371910,10.61285,12.19458,701.266296,11.687771,742.743408,12.379057
2,2025-04-12,Biu,2025-04-17 00:00:00+00:00,3,26.679255,42106.542969,0.0,0.0,25.870001,38.973499,...,39.630596,0.0,0.0,300.154419,10.61285,12.19458,701.775696,11.696262,743.295532,12.388259
3,2025-04-12,Biu,2025-04-13 00:00:00+00:00,0,18.847942,41872.453125,0.0,0.0,28.260000,36.523499,...,34.716145,0.0,0.0,58.123203,10.61285,12.19458,697.874207,11.631237,741.055908,12.350932
4,2025-04-12,Biu,2025-04-18 00:00:00+00:00,2,23.508938,42136.835938,0.0,0.0,27.809999,39.573498,...,39.437950,0.0,0.0,56.246418,10.61285,12.19458,702.280579,11.704677,743.846497,12.397442
5,2025-04-12,Biu,2025-04-12 00:00:00+00:00,3,21.073042,39960.453125,0.0,0.0,26.219999,36.323498,...,34.464893,0.0,0.0,34.963619,10.61285,12.19458,666.007568,11.100126,740.483826,12.341397


#### 🌍 **2.5 FORECAST HOURLY**
**🧹 Cleaning & Transformation Logic:**

We will apply the following transformations:

- **Drop columns**
- **Rename columns**
- **Convert data types** 
- **Order the table**


>We will keep the same logic of the bronze layer and upsert the data. 

In [14]:
# Load DeltaTable 
name_folder = 'Data/Bronze/OpenMeteo/Forecast/Hourly'
my_dt = DeltaTable(name_folder).to_pandas()

# Drop unnecessary column
my_dt.drop(columns=['__index_level_0__'], inplace=True)

# Convert date columns to datetime format
my_dt['Requested_Date'] = pd.to_datetime(my_dt['Requested_Date'], format='%Y-%m-%d')
my_dt['Forecast_Date'] = pd.to_datetime(my_dt['Forecast_Date'], format='%Y-%m-%d')
# Change column types
my_dt['longitude'] = my_dt['longitude'].astype(float)
my_dt['latitude'] = my_dt['latitude'].astype(float)
my_dt["weather_code"] = my_dt["weather_code"].astype(int)
my_dt["weather_code"] = my_dt["weather_code"].astype(str)
my_dt['weather_code'] = my_dt['weather_code'].str.strip()


# Rename columns to include units of measure
remapping = {
    "soil_moisture_27_to_81cm": "soil_moisture_27_to_81cm_m3m3",
    "soil_moisture_9_to_27cm": "soil_moisture_9_to_27cm_m3m3",
    "soil_moisture_3_to_9cm": "soil_moisture_3_to_9cm_m3m3",
    "soil_moisture_1_to_3cm": "soil_moisture_1_to_3cm_m3m3",
    "soil_moisture_0_to_1cm": "soil_moisture_0_to_1cm_m3m3",
    "soil_temperature_54cm": "soil_temperature_54cm_C",
    "soil_temperature_18cm": "soil_temperature_18cm_C",
    "soil_temperature_6cm": "soil_temperature_6cm_C",
    "soil_temperature_0cm": "soil_temperature_0cm_C",
    "temperature_180m": "temperature_180m_C",
    "temperature_120m": "temperature_120m_C",
    "temperature_80m": "temperature_80m_C",
    "temperature_2m": "temperature_2m_C",
    "wind_gusts_10m": "wind_gusts_10m_kmh",
    "wind_direction_180m": "wind_direction_180m_deg",
    "wind_direction_120m": "wind_direction_120m_deg",
    "wind_direction_80m": "wind_direction_80m_deg",
    "wind_direction_10m": "wind_direction_10m_deg",
    "wind_speed_180m": "wind_speed_180m_kmh",
    "wind_speed_120m": "wind_speed_120m_kmh",
    "wind_speed_80m": "wind_speed_80m_kmh",
    "wind_speed_10m": "wind_speed_10m_kmh",
    "vapour_pressure_deficit": "vapour_pressure_deficit_kPa",
    "et0_fao_evapotranspiration_mm": "et0_fao_evapotranspiration_mm",
    "evapotranspiration": "evapotranspiration_mm",
    "visibility": "visibility_m",
    "cloud_cover_high": "cloud_cover_high_inPercent",
    "cloud_cover_mid": "cloud_cover_mid_inPercent",
    "cloud_cover_low": "cloud_cover_low_inPercent",
    "cloud_cover": "cloud_cover_inPercent",
    "surface_pressure": "surface_pressure_hPa",
    "pressure_msl": "pressure_msl_hPa",
    "weather_code": "weather_code",
    "snow_depth": "snow_depth_m",
    "snowfall": "snowfall_cm",
    "showers": "showers_mm",
    "rain": "rain_mm",
    "precipitation": "precipitation_mm",
    "precipitation_probability": "precipitation_probability_inPercent",
    "apparent_temperature": "apparent_temperature_C",
    "dew_point_2m": "dew_point_2m_C",
    "relative_humidity_2m": "relative_humidity_2m_inPercent",
    "Requested_Date": 'requested_date',
    'City': 'city',
    'Forecast_Date': 'forecast_date',
    'Forecast_Hour': 'forecast_hour'
}
my_dt = my_dt.rename(columns=remapping)


#Ordering columns
my_dt = order_table(my_dt, ['requested_date','city','forecast_date','forecast_hour'])
# Display DataFrame preview

#delete duplicates
#my_dt.drop_duplicates(keep='first')

#my_dt.info()
my_dt.head(6)

,requested_date,city,forecast_date,forecast_hour,longitude,latitude,soil_moisture_27_to_81cm_m3m3,soil_moisture_9_to_27cm_m3m3,soil_moisture_3_to_9cm_m3m3,soil_moisture_1_to_3cm_m3m3,...,snow_depth_m,snowfall_cm,showers_mm,rain_mm,precipitation_mm,precipitation_probability_inPercent,apparent_temperature_C,dew_point_2m_C,relative_humidity_2m_inPercent,temperature_2m_C
0,2025-04-11,Barcelona,2025-04-10,00:00,2.15899,41.38879,0.318,0.287,0.272,0.267,...,0.0,0.0,0.0,0.0,0.0,8.0,13.368288,12.337662,95.0,13.120999
1,2025-04-11,Barcelona,2025-04-10,01:00,2.15899,41.38879,0.318,0.287,0.272,0.267,...,0.0,0.0,0.0,0.0,0.0,0.0,12.995403,11.978119,94.0,12.921000
2,2025-04-11,Barcelona,2025-04-10,02:00,2.15899,41.38879,0.318,0.287,0.273,0.268,...,0.0,0.0,0.0,0.0,0.0,0.0,12.747696,11.779591,94.0,12.721000
3,2025-04-11,Barcelona,2025-04-10,03:00,2.15899,41.38879,0.318,0.287,0.273,0.268,...,0.0,0.0,0.0,0.0,0.0,0.0,13.091465,11.928487,94.0,12.870999
4,2025-04-11,Barcelona,2025-04-10,04:00,2.15899,41.38879,0.318,0.287,0.273,0.269,...,0.0,0.0,0.0,0.0,0.0,0.0,13.564077,11.949058,92.0,13.221000
5,2025-04-11,Barcelona,2025-04-10,05:00,2.15899,41.38879,0.318,0.287,0.273,0.269,...,0.0,0.0,0.0,0.0,0.0,3.0,14.038681,12.394598,92.0,13.671000


In [15]:
#IN ORDER TO UPDATE THE TYPE OF THE COLUMNS
#name_folder = 'Data/Silver/OpenMeteo/Forecast/Hourly'
#write_deltalake(name_folder,my_dt,mode='overwrite',schema_mode='overwrite')

In [16]:
#STORING THE DATA
name_folder = 'Data/Silver/OpenMeteo/Forecast/Hourly'
partition_cols = ["requested_date"]
predicate = """target.requested_date = source.requested_date  AND target.city = source.city AND target.forecast_date = source.forecast_date and target.forecast_hour = source.forecast_hour """

upsert_data_as_delta(my_dt,name_folder,predicate= predicate, partition_cols=partition_cols, layer = 'Silver', source= 'open-meteo-forecast-hourly', author ='Augustin')

# Verifying the data of the bronze layer
my_dt = DeltaTable(name_folder).to_pandas()
my_dt.head(6)

,requested_date,city,forecast_date,forecast_hour,longitude,latitude,soil_moisture_27_to_81cm_m3m3,soil_moisture_9_to_27cm_m3m3,soil_moisture_3_to_9cm_m3m3,soil_moisture_1_to_3cm_m3m3,...,snow_depth_m,snowfall_cm,showers_mm,rain_mm,precipitation_mm,precipitation_probability_inPercent,apparent_temperature_C,dew_point_2m_C,relative_humidity_2m_inPercent,temperature_2m_C
0,2025-04-12,Biu,2025-04-12,09:00,12.19458,10.61285,0.156,0.128,0.111,0.087,...,0.0,0.0,0.0,0.0,0.0,0.0,25.565151,1.661153,16.0,30.273499
1,2025-04-12,Biu,2025-04-12,12:00,12.19458,10.61285,0.156,0.128,0.110,0.085,...,0.0,0.0,0.0,0.0,0.0,0.0,34.024841,1.652948,12.0,35.373501
2,2025-04-12,Biu,2025-04-12,14:00,12.19458,10.61285,0.156,0.127,0.110,0.083,...,0.0,0.0,0.0,0.0,0.0,0.0,34.098835,1.169002,11.0,36.323498
3,2025-04-12,Biu,2025-04-13,02:00,12.19458,10.61285,0.156,0.127,0.108,0.077,...,0.0,0.0,0.0,0.0,0.0,0.0,21.336058,3.116406,25.0,24.423500
4,2025-04-12,Biu,2025-04-13,19:00,12.19458,10.61285,0.155,0.125,0.105,0.073,...,0.0,0.0,0.0,0.0,0.0,0.0,25.649120,1.178905,17.0,28.623499
5,2025-04-12,Biu,2025-04-13,20:00,12.19458,10.61285,0.155,0.125,0.105,0.072,...,0.0,0.0,0.0,0.0,0.0,0.0,24.750059,1.836748,19.0,27.523499


#### 🌍 **2.6 HISTORICAL DAILY**
**🧹 Cleaning & Transformation Logic:**

We will apply the following transformations:

- **Drop columns**
- **Rename columns**
- **Compute new columns** (duration in hours and in minutes)
- **Convert data types** 
- **Order the table**


>We will keep the same logic of the bronze layer and upsert the data. 

In [17]:
# Load DeltaTable 
name_folder = 'Data/Bronze/OpenMeteo/Historical/Daily'
my_dt = DeltaTable(name_folder).to_pandas()

# Drop columns
my_dt.drop(columns=['__index_level_0__', 'sunrise', 'sunset', 'uv_index_clear_sky_max', 
                    'uv_index_max', 'precipitation_probability_max'], inplace=True)

# Rename columns to include units of measure
remapping = {
    "temperature_2m_max": "temperature_2m_max_C",
    "temperature_2m_min": "temperature_2m_min_C",
    "apparent_temperature_max": "apparent_temperature_max_C",
    "apparent_temperature_min": "apparent_temperature_min_C",
    "precipitation_sum_mm": "precipitation_sum_mm",
    "rain_sum": "rain_sum_mm",
    "showers_sum": "showers_sum_mm",
    "snowfall_sum": "snowfall_sum_cm",
    "precipitation_hours": "precipitation_hours_h",
    "sunshine_duration": "sunshine_duration_seconds",
    "daylight_duration": "daylight_duration_seconds",
    "wind_speed_10m_max": "wind_speed_10m_max_kmh",
    "wind_gusts_10m_max": "wind_gusts_10m_max_kmh",
    "wind_direction_10m_dominant": "wind_direction_10m_dominant_deg",
    "shortwave_radiation_sum": "shortwave_radiation_sum_MJm2",
    "et0_fao_evapotranspiration": "et0_fao_evapotranspiration_mm",
    "Requested_Date": 'requested_date',
    'City': 'city',
    'Historical_Date': 'historical_date',
    'Historical_Year': 'historical_year',
    'Historical_Month': 'historical_month',
    'Historical_Day': 'historical_day'
}
my_dt = my_dt.rename(columns=remapping)

# Convert date columns and calculate durations in minutes and hours
my_dt['historical_date'] = pd.to_datetime(my_dt['historical_date'], format='%Y-%m-%d')
my_dt['sunshine_duration_minutes'] = my_dt['sunshine_duration_seconds'] / 60
my_dt['sunshine_duration_hours'] = my_dt['sunshine_duration_seconds'] / 3600
my_dt['daylight_duration_minutes'] = my_dt['daylight_duration_seconds'] / 60
my_dt['daylight_duration_hours'] = my_dt['daylight_duration_seconds'] / 3600

# Change column types
my_dt['longitude'] = my_dt['longitude'].astype(float)
my_dt['latitude'] = my_dt['latitude'].astype(float)
my_dt["weather_code"] = my_dt["weather_code"].astype(int)
my_dt["weather_code"] = my_dt["weather_code"].astype(str)
my_dt['weather_code'] = my_dt['weather_code'].str.strip()


#Ordering columns
my_dt = order_table(my_dt, ['city','historical_year','historical_month','historical_day'])

# Display DataFrame preview
#my_dt.info()
#my_dt.head(6)

In [18]:
#IN ORDER TO UPDATE THE TYPE OF THE COLUMNS
#name_folder = 'Data/Silver/OpenMeteo/Historical/Daily'
#write_deltalake(name_folder,my_dt,mode='overwrite',schema_mode='overwrite')

In [19]:
#STORING THE DATA
name_folder = 'Data/Silver/OpenMeteo/Historical/Daily'
predicate = """target.city = source.city AND target.historical_date = source.historical_date"""
partition_cols = ["historical_year"]

save_new_data_as_delta(my_dt,name_folder,predicate= predicate, partition_cols=partition_cols, layer = 'Silver', source= 'open-meteo-historical-daily', author ='Augustin')


# Verifying the data of the silver layer
my_dt = DeltaTable(name_folder).to_pandas()
my_dt.head()

,city,historical_date,historical_year,historical_month,historical_day,longitude,latitude,wind_direction_10m_dominant_deg,precipitation_hours_h,precipitation_sum,...,sunshine_duration_seconds,apparent_temperature_min_C,weather_code,sunshine_duration_minutes,sunshine_duration_hours,daylight_duration_minutes,daylight_duration_hours,warm_enough,too_cold,sufficient_sunshine
0,Barcelona,2013-01-18,2013,01,Fri,2.15899,41.38879,245.865112,9.0,2.4,...,18306.171875,5.373896,51,305.102875,5.085048,576.679810,9.611330,None,None,None
1,Barcelona,2013-01-26,2013,01,Sat,2.15899,41.38879,35.972466,4.0,1.2,...,22572.945312,0.124769,53,376.215759,6.270263,592.010498,9.866841,None,None,None
2,Barcelona,2013-01-24,2013,01,Thu,2.15899,41.38879,299.295288,3.0,0.6,...,29427.308594,2.775758,51,490.455139,8.174253,587.966187,9.799437,None,None,None
3,Barcelona,2013-02-15,2013,02,Fri,2.15899,41.38879,211.914871,0.0,0.0,...,34980.632812,4.436029,3,583.010559,9.716843,639.140747,10.652346,None,None,None
4,Barcelona,2013-02-03,2013,02,Sun,2.15899,41.38879,327.231506,0.0,0.0,...,33073.597656,-0.236782,1,551.226624,9.187111,609.381714,10.156362,None,None,None


**COMPACTING THE DATA**

In [20]:
name_folder = 'Data/_meta/metadata_table'
my_dt = DeltaTable(name_folder).to_pandas()
#Call to my own function to compact the data
compact_all_silver_tables(my_dt, layer_filter='Silver')

✅ Compacté : Data/Silver/OpenMeteo/Historical/Daily
✅ Compacté : Data/Silver/OpenMeteo/Forecast/Hourly
✅ Compacté : Data/Silver/OpenMeteo/Forecast/Daily
✅ Compacté : Data/Silver/OpenMeteo/Current
✅ Compacté : Data/Silver/OpenMeteo/Others/WeatherCode
✅ Compacté : Data/Silver/OpenMeteo/Others/Geolocation


**📊 CHECK SILVER TABLE STATS AND COMPARING WITH BRONZE: Rows, Nulls, Duplicates**

In [21]:
#Checking silver layer
name_folder = 'Data/_meta/metadata_table'
my_dt = DeltaTable(name_folder).to_pandas()
my_dt = my_dt[my_dt['layer']=='Silver']
my_dt.head(10)

,table_path,table_name,layer,total_rows,rows_with_nulls,rows_duplicated,columns,dtypes,delta_table_size_MB,file_count,updated_at,created_at,source,author
0,Data/Silver/OpenMeteo/Historical/Daily,Daily,Silver,69049,63939,0,"[""city"", ""historical_date"", ""historical_year"",...","{""city"": ""object"", ""historical_date"": ""datetim...",5.28,21,2025-04-12 23:46:04.460107,2025-04-12 15:40:16.240528,open-meteo-historical-daily,Augustin
1,Data/Silver/OpenMeteo/Forecast/Hourly,Hourly,Silver,4632,0,0,"[""requested_date"", ""city"", ""forecast_date"", ""f...","{""requested_date"": ""datetime64[us]"", ""city"": ""...",33.78,51,2025-04-12 23:46:04.107318,2025-04-12 13:40:22.692732,open-meteo-forecast-hourly,Augustin
2,Data/Silver/OpenMeteo/Forecast/Daily,Daily,Silver,231,0,0,"[""requested_date"", ""city"", ""forecast_day"", ""we...","{""requested_date"": ""datetime64[us]"", ""city"": ""...",0.15,13,2025-04-12 23:46:03.897487,2025-04-12 13:37:27.503434,open-meteo-forecast-daily,Augustin
3,Data/Silver/OpenMeteo/Current,Current,Silver,19,0,0,"[""date"", ""time"", ""city"", ""longitude"", ""latitud...","{""date"": ""datetime64[us]"", ""time"": ""object"", ""...",0.14,28,2025-04-12 23:46:03.723835,2025-04-12 13:33:12.200944,open-meteo-current,Augustin
4,Data/Silver/OpenMeteo/Others/WeatherCode,WeatherCode,Silver,28,0,0,"[""weather_code"", ""description""]","{""weather_code"": ""object"", ""description"": ""obj...",0.03,28,2025-04-12 23:46:03.633625,2025-04-12 12:49:23.951869,open-meteo-weathercode,Augustin
5,Data/Silver/OpenMeteo/Others/Geolocation,Geolocation,Silver,37,0,0,"[""id"", ""name"", ""latitude"", ""longitude"", ""eleva...","{""id"": ""int64"", ""name"": ""object"", ""latitude"": ...",0.04,8,2025-04-12 23:46:03.542779,2025-04-12 13:28:20.780053,open-meteo-geoloc,Augustin


In [22]:
#Comparating silver and bronze
name_folder = 'Data/_meta/metadata_table'
my_dt = DeltaTable(name_folder).to_pandas()
my_dt = my_dt[(my_dt['layer'] == 'Bronze') | (my_dt['layer'] == 'Silver')]
row_counts_per_table = pd.DataFrame({
    "layer":my_dt["layer"],
    "table_name": my_dt["table_name"],
    "table_path": my_dt["table_path"],
    "total_rows": my_dt['total_rows'],
    "rows_with_at_least_one_nulls":my_dt['rows_with_nulls'],
    "rows_duplicated":my_dt['rows_duplicated']
})

row_counts_per_table.sort_values(by='table_path').head(30)

,layer,table_name,table_path,total_rows,rows_with_at_least_one_nulls,rows_duplicated
9,Bronze,Current,Data/Bronze/OpenMeteo/Current,19,0,0
8,Bronze,Daily,Data/Bronze/OpenMeteo/Forecast/Daily,231,0,0
7,Bronze,Hourly,Data/Bronze/OpenMeteo/Forecast/Hourly,4632,0,0
6,Bronze,Daily,Data/Bronze/OpenMeteo/Historical/Daily,69049,69049,0
11,Bronze,Geolocation,Data/Bronze/OpenMeteo/Others/Geolocation,37,36,0
10,Bronze,WeatherCode,Data/Bronze/OpenMeteo/Others/WeatherCode,13,0,0
3,Silver,Current,Data/Silver/OpenMeteo/Current,19,0,0
2,Silver,Daily,Data/Silver/OpenMeteo/Forecast/Daily,231,0,0
1,Silver,Hourly,Data/Silver/OpenMeteo/Forecast/Hourly,4632,0,0
0,Silver,Daily,Data/Silver/OpenMeteo/Historical/Daily,69049,63939,0


In [23]:
export_metadata_to_excel(layer='Silver')

✅ Métadonnées exportées avec succès dans : logs/2025-04-12/silver_metadata_20h46.xlsx
